<a href="https://colab.research.google.com/github/jlmarrugom/covid/blob/main/AnimalesDomesticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium  #needed for interactive map
from folium.plugins import HeatMap

In [ ]:
pets =pd.read_excel('/content/drive/MyDrive/Covid UC/BASE DE DATOS ANIMALES DOMÉSTICOS_29-01-2021.xlsx',sheet_name='Hoja1')
pets['Conteo']=1
pets.head(3)

,Nro,FECHA TOMA DE MUESTRA D/M/A,CÓDIGO CAMPO,CODIGO IIBT,NOMBRE DEL PROPIETARIO,ID DEL ANIMAL,ESPECIE,SEXO,EDAD APROX.,HISOPADO OROFARINGEO,HISOPADO RECTAL,SANGRE,FECHA INICIO DE SINTOMAS PROPIETARIO,TELEFONO PROPIETARIO,MUNICIPIO,DIRECCIÓN,COORDENADAS,RESULTADO PCR,Conteo
0,1,2020-09-15 00:00:00,CAT001,D01,LILIA HOYOS,TOM,FELINO,M,3 M,SI,NO,SI,NaT,NO REGISTRA,SAHAGÚN,NaN,NaN,NEGATIVO,1
1,2,2020-09-15 00:00:00,CAT002,D02,BEATRIZ HERRERA,MARIA PAULA,FELINO,H,4 A,SI,NO,SI,NaT,NO REGISTRA,SAHAGÚN,NaN,NaN,NEGATIVO,1
2,3,2020-09-15 00:00:00,CAT003,D03,ANA KARINA OYOLA,ALEJANDRA,FELINO,H,3 M,SI,NO,SI,NaT,NO REGISTRA,SAHAGÚN,NaN,NaN,NEGATIVO,1


In [ ]:
pets['MUNICIPIO'].value_counts(),pets['RESULTADO PCR'].value_counts()

(CERETE          42
 PLANETA RICA    39
 SAHAGÚN         29
 LORICA          25
 TIERRALTA       23
 SAN ANTERO      20
 MONTERIA        16
 MONTELIBANO     13
 Name: MUNICIPIO, dtype: int64, NEGATIVO    206
 POSITIVO      1
 Name: RESULTADO PCR, dtype: int64)

In [ ]:
#@title Georeferenciación
pets['lat']=0
pets['lon']=0
pets['lat'].loc[pets['MUNICIPIO']=='LORICA'] = 9.2394583
pets['lon'].loc[pets['MUNICIPIO']=='LORICA'] = -75.8139786

pets['lat'].loc[pets['MUNICIPIO']=='PLANETA RICA'] = 8.4076739 
pets['lon'].loc[pets['MUNICIPIO']=='PLANETA RICA'] = -75.5840456 

pets['lat'].loc[pets['MUNICIPIO']=='TIERRALTA '] = 8.1717342
pets['lon'].loc[pets['MUNICIPIO']=='TIERRALTA '] = -76.059376

pets['lat'].loc[pets['MUNICIPIO']=='SAHAGÚN'] = 8.9472964
pets['lon'].loc[pets['MUNICIPIO']=='SAHAGÚN'] = -75.4434972

pets['lat'].loc[pets['MUNICIPIO']=='MONTELIBANO'] = 7.9800534
pets['lon'].loc[pets['MUNICIPIO']=='MONTELIBANO'] = -75.4167198

pets['lat'].loc[pets['MUNICIPIO']=='CERETE'] = 8.8852282
pets['lon'].loc[pets['MUNICIPIO']=='CERETE'] = -75.7922421

pets['lat'].loc[pets['MUNICIPIO']=='SAN ANTERO'] = 9.373016
pets['lon'].loc[pets['MUNICIPIO']=='SAN ANTERO'] = -75.7595056

pets['lat'].loc[pets['MUNICIPIO']=='MONTERIA'] = 8.7558921
pets['lon'].loc[pets['MUNICIPIO']=='MONTERIA'] = -75.887029

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
#@title Tabla para Mapa
df = pets.copy(deep=True)
# max_amount = float(df['export_val'].max())
df = df[['lat','lon','MUNICIPIO']].groupby('MUNICIPIO').max()
df = df.merge(pets[['RESULTADO PCR','MUNICIPIO']].loc[pets['RESULTADO PCR']=='POSITIVO'].groupby(['MUNICIPIO']).count() ,how='outer',on='MUNICIPIO')
df = df.merge(pets[['RESULTADO PCR','MUNICIPIO']].groupby(['MUNICIPIO']).count() ,how='outer',on='MUNICIPIO')
df = df.merge(pets[['ESPECIE','MUNICIPIO']].groupby(['MUNICIPIO']).agg(lambda x:x.value_counts().index[0]) ,how='inner',on='MUNICIPIO')
df = df.merge(pets[['SEXO','MUNICIPIO']].groupby(['MUNICIPIO']).agg(lambda x:x.value_counts().index[0]) ,how='inner',on='MUNICIPIO')
df = df.merge(pets[['EDAD APROX.','MUNICIPIO']].groupby(['MUNICIPIO']).agg(lambda x:x.value_counts().index[0]) ,how='inner',on='MUNICIPIO')


#df = df.merge(pets[['RESULTADO PCR','MUNICIPIO']].loc[pets['RESULTADO PCR']=='POSITIVO'].groupby(['MUNICIPIO']).agg(lambda x:x.value_counts().index[0])

df = df.reset_index()
df

,MUNICIPIO,lat,lon,RESULTADO PCR_x,RESULTADO PCR_y,ESPECIE,SEXO,EDAD APROX.
0,CERETE,8.885228,-75.792242,NaN,42,FELINO,H,NR
1,LORICA,9.239458,-75.813979,NaN,25,FELINO,M,2 A
2,MONTELIBANO,7.980053,-75.416720,NaN,13,FELINO,H,1 A
3,MONTERIA,8.755892,-75.887029,1.0,16,FELINO,H,1 A
4,PLANETA RICA,8.407674,-75.584046,NaN,39,FELINO,M,2 A
5,SAHAGÚN,8.947296,-75.443497,NaN,29,FELINO,M,3 A
6,SAN ANTERO,9.373016,-75.759506,NaN,20,FELINO,M,3 M
7,TIERRALTA,8.171734,-76.059376,NaN,23,FELINO,M,1 A


In [ ]:
#@title #Conteo por Municipio
folium_hmap = folium.Figure(width=1000, height=500)
m = folium.Map(location=[8.3344713,-75.6666238],
                         width='100%',
                         height='100%',
                         zoom_start=8,#Por defecto es 10
                         tiles="OpenStreetMap").add_to(folium_hmap)

#folium_hmap.fit_bounds([[6.4, -78], [10.4, -73]])
data = df
for i in range(0,len(data)):
   folium.Circle(
      location=[data.iloc[i]['lat'], data.iloc[i]['lon']],
      popup=str(data.iloc[i]['MUNICIPIO'])+
      '\nTotal de Pruebas Realizadas: \n'+
      str(data.iloc[i]['RESULTADO PCR_y'])+
      '\nPositivas: \n'+
      str(data.iloc[i]['RESULTADO PCR_x']),
      radius=float(data.iloc[i]['RESULTADO PCR_y'])*300,
      color='blue',
      fill=True,
      fill_color='blue'
   ).add_to(m)

folium_hmap